<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data. 

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

Some familiarity with Python is helpful. This notebook is compatible Watson Studio Desktop and Python 3.6.

## Learning goals

In this notebook you learn how to:

-  Set up a Watson Machine Learning Server.
-  Deploy the SPSS model online.
-  Use the deployed model to score data.


## Contents

1.	[Set up the environment](#setup)
2.	[Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository](#scoring)
3.	[Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup task:

-  Install <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Server</a>.

Install <a href="https://pypi.org/project/watson-machine-learning-client-V4/" target="_blank" rel="noopener no referrer">watson-machine-learning-client</a> package (not required on Watson Studio).

**Note:** `watson-machine-learning-client` documentation can be found <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
!pip install --upgrade watson-machine-learning-client-V4==1.0.58

Requirement already up-to-date: watson-machine-learning-client-V4==1.0.58 in /cc-home/_global_/python-3.6 (1.0.58)


**Connect to the Watson Machine Learning Server using the Python client**<br><br>

<div class="alert-block alert-info"><br>
To install the Watson Machine Learning Server, follow <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">these documentation steps</a>. To connect to the WML server and find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-connect.html" target="_blank" rel="noopener no referrer">Documentation</a>.<br><br>
</div>

**Action**: Enter your WML Server credentials in the following cell.

In [2]:
wml_credentials={
    'url': '***',
    'username': '***',
    'password': '***',
    'instance_id': 'icp',
    'version': '***'
}

Import the `watson-machine-learning-client` module and authenticate the service instance.

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.58'

<a id="scoring"></a>
## 2. Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository

In this section, you will learn how to:
- [2.1 Download and save an SPSS model](#save)
- [2.2 Create an online deployment](#create)
- [2.3 Score a test data record](#score)

### 2.1 Download and save an SPSS model<a id="save"></a>

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

**Note:** You may need to install the `wget` package. To install the `wget` package, run the following command. 

In [6]:
!pip install --upgrade wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=45477591e11495bfaa61b6b915107e78085d59601676af58007df8ccc2667719
  Stored in directory: /home/wsuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [7]:
# Download the sample SPSS model.
import os
import wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


In [8]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

In [9]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 01e8c92c-171f-430d-b29a-49f7d2d3243f


In [10]:
# Set the space to use it in subsequent actions
client.set.default_space(space_uid)

'SUCCESS'

Get a list of stored models in the WML repository.

In [11]:
client.repository.list_models()

----  ----  -------  ----
GUID  NAME  CREATED  TYPE
----  ----  -------  ----


Get a list of deployed models.

In [12]:
client.deployments.list()

----  ----  -----  -------  -------------
GUID  NAME  STATE  CREATED  ARTIFACT_TYPE
----  ----  -----  -------  -------------


Save the SPSS sample model in your WML repository. First, you need to create the model metadata to store in the repository.

In [13]:
# Save the SPSS model to the WML repository.
# Model Metadata.
meta_props={
    client.repository.ModelMetaNames.NAME: "SPSS model for Churn prediction",
    client.repository.ModelMetaNames.RUNTIME_UID: "spss-modeler_18.1",
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.1",
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

You need the model uid to create the deployment. You can extract the model uid from the saved model details and use it in the next section to create the deployment.

In [14]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=meta_props)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = d98b2f65-9932-4017-93c7-cfef0b92fc32


Get the saved model metadata from WML.

In [15]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'name': 'SPSS model for Churn prediction',
            'runtime': {'href': '/v4/runtimes/spss-modeler_18.1'},
            'space': {'href': '/v4/spaces/01e8c92c-171f-430d-b29a-49f7d2d3243f'},
            'type': 'spss-modeler_18.1'},
 'metadata': {'created_at': '2019-12-20T01:02:44.002Z',
              'guid': 'd98b2f65-9932-4017-93c7-cfef0b92fc32',
              'href': '/v4/models/d98b2f65-9932-4017-93c7-cfef0b92fc32?space_id=01e8c92c-171f-430d-b29a-49f7d2d3243f',
              'id': 'd98b2f65-9932-4017-93c7-cfef0b92fc32',
              'modified_at': '2019-12-20T01:02:45.002Z',
              'owner': '999'}}


You can list all stored models using the `list_models` method.

In [16]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   TYPE
d98b2f65-9932-4017-93c7-cfef0b92fc32  SPSS model for Churn prediction  2019-12-20T01:02:44.002Z  spss-modeler_18.1
------------------------------------  -------------------------------  ------------------------  -----------------


### 2.2 Create an online deployment<a id="create"></a>

Using the code below to create an online deployment.

In [17]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Sample SPSS model deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [18]:
# Create the deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'd98b2f65-9932-4017-93c7-cfef0b92fc32' started

#######################################################################################


initializing....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='226d2f70-dcf4-45d7-a7a5-738d7a31816e'
------------------------------------------------------------------------------------------------




Get the list of all deployments.

In [19]:
# List the deployments.
client.deployments.list()

------------------------------------  ----------------------------  -----  ------------------------  -------------
GUID                                  NAME                          STATE  CREATED                   ARTIFACT_TYPE
226d2f70-dcf4-45d7-a7a5-738d7a31816e  Sample SPSS model deployment  ready  2019-12-20T01:02:48.701Z  model
------------------------------------  ----------------------------  -----  ------------------------  -------------


Now, you can check details of your deployments.

In [20]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 226d2f70-dcf4-45d7-a7a5-738d7a31816e


In [21]:
# Deployment details.
print(client.deployments.get_details(deployment_uid))

{'metadata': {'parent': {'href': ''}, 'guid': '226d2f70-dcf4-45d7-a7a5-738d7a31816e', 'modified_at': '', 'created_at': '2019-12-20T01:02:48.701Z', 'href': '/v4/deployments/226d2f70-dcf4-45d7-a7a5-738d7a31816e'}, 'entity': {'name': 'Sample SPSS model deployment', 'custom': {}, 'online': {}, 'description': '', 'space': {'href': '/v4/spaces/01e8c92c-171f-430d-b29a-49f7d2d3243f'}, 'status': {'state': 'ready', 'online_url': {'url': 'https://wmlserver-svt.ml.test.cloud.ibm.com:31843/v4/deployments/226d2f70-dcf4-45d7-a7a5-738d7a31816e/predictions'}}, 'asset': {'href': '/v4/models/d98b2f65-9932-4017-93c7-cfef0b92fc32?space_id=01e8c92c-171f-430d-b29a-49f7d2d3243f'}, 'auto_redeploy': False}}


### 2.3 Score a test data record<a id="score"></a>

Prepare the scoring payload with values to score against the model.

In [22]:
# Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields':['customerID','gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges','TotalCharges','Churn','SampleWeight'],
        'values':[['3638-WEABW','Female',0,'Yes','No',58,'Yes','Yes','DSL','No','Yes','No','Yes','No','No','Two year','Yes','Credit card (automatic)',59.9,3505.1,'No',2.768]]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['customerID',
                            'gender',
                            'SeniorCitizen',
                            'Partner',
                            'Dependents',
                            'tenure',
                            'PhoneService',
                            'MultipleLines',
                            'InternetService',
                            'OnlineSecurity',
                            'OnlineBackup',
                            'DeviceProtection',
                            'TechSupport',
                            'StreamingTV',
                            'StreamingMovies',
                            'Contract',
                            'PaperlessBilling',
                            'PaymentMethod',
                            'MonthlyCharges',
                            'TotalCharges',
                            'Churn',
                            'SampleWeight'],
                 'values': [['3638-WEABW',
 

Use the client `score` method to score test data about a customer.

In [23]:
# Perform prediction and display the result.
job_details = client.deployments.score(deployment_uid, job_payload)
pprint(job_details)

{'predictions': [{'fields': ['customerID',
                             'Churn',
                             'Predicted Churn',
                             'Probability of Churn'],
                  'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}]}


As you can see, this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="summary"></a>
## 3. Summary     

You successfully completed this notebook! 

You learned how to create a regression model and use Watson Machine Learning to create and deploy a model. 

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success"><a id="resources"></a>

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/overview.html" target="_blank" rel="noopener noreferrer">Watson Machine Learning Server</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 <li> <a href="https://matplotlib.org" target="_blank" rel="noopener noreferrer">Matplotlib: Python plotting</a></li>
 <li> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html" target="_blank" rel="noopener noreferrer">scikit-learn: Grid Search</a></li>
 </div>

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2017-2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>